<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# **Procesamiento de Lenguaje Natural**
# **Desafío 2: Custom embeddings**

> **Carrera de Especialización en Inteligencia Artificial, Facultad de Ingeniería**
>
> **Universidad de Buenos Aires, Junio de 2024**
>
> Edgar David Guarin Castro (davidg@marketpsychdata.com)

En el presente trabajo se busca usar documentos / corpus para crear embeddings de palabras basado en el contexto de noticias financieras. Para ello se utilizará el dataset de [notícias financieras de Reuters](https://github.com/duynht/financial-news-dataset) con el fin de generar los vectores de palabras cuya forma será definida en función de cómo las palabras son usadas en el contexto.

## **0. Importando librerías**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import multiprocessing
from gensim.models import Word2Vec
import numpy as np
import os
import platform
import plotly.express as px

from tensorflow.keras.preprocessing.text import text_to_word_sequence
from gensim.models.callbacks import CallbackAny2Vec
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNEimport plotly.graph_objects as go

## **1. Cargando los datos**

El dataset de noticias financieras contiene 109,110 noticias de Reuters, recopiladas y utilizadas por primera vez en una investigación de [Ding et al. (2014)](https://emnlp2014.org/papers/pdf/EMNLP2014148.pdf). El dataset se ha utilizado en otros estudios de predicción de movimientos del precio de acciones basados en eventos estructurados.

Despues de clonar el repositório, una carpeta llamada *financial-news-dataset* con todos los artículos, es creada en el directorio local:

In [2]:
#-----------------
# Clonando el repositorio
#-----------------
!git clone https://github.com/duynht/financial-news-dataset.git

Cloning into 'financial-news-dataset'...
remote: Enumerating objects: 109112, done.
remote: Total 109112 (delta 0), reused 0 (delta 0), pack-reused 109112
Receiving objects: 100% (109112/109112), 162.92 MiB | 14.21 MiB/s, done.
Resolving deltas: 100% (1046/1046), done.
Updating files: 100% (106523/106523), done.


Los artículos del corpus son cargados a un dataframe y se usa la mitad de los artículos en este caso para agilizar los cálculos:

In [3]:
#-----------------
# Función para leer todas las noticias
#-----------------
def read_news(directory):
    news = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            with open(file_path, 'r', encoding='latin-1') as f:
                news.append(f.read())
    return news

#-----------------
# Leyendo las noticias del directorio ReutersNews106521
#-----------------
news = read_news('financial-news-dataset/ReutersNews106521')

#-----------------
# Creando un DataFrame con la mitad de las noticias
#-----------------
df = pd.DataFrame(news, columns=['article'])

df = df[:int(df.shape[0]/2)]

df

,article
0,   Bud1                                  ...
1,"-- Hitachi, GE boost alliance in nuclear power..."
2,"-- Volvo to cut 1,000 staff at Virginia plant\..."
3,-- European banks hiding full pension obligati...
4,"-- Hitachi, GE to form joint nuclear power ven..."
...,...
53255,-- China's exporters need not fear freer yuan:...
53256,-- Economic growth trimmed on consumer and bus...
53257,-- UAL and Continental pilot negotiations hit ...
53258,-- U.S. asks court to keep deepwater drilling ...


In [4]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 53260


## **2. Preprocesamiento**

A continuación, se recorren todas las filas del dataframe y se transforman las noticias en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también):

In [5]:
#-----------------
# Lista para guardar los tokens
#-----------------
sentence_tokens = []

for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

#-----------------
# Revisando el resultado
#-----------------
sentence_tokens[:2]

/var/folders/sz/sjmjp0xj2r35nk_bcvzdc0wc0000gn/T/ipykernel_28930/3611914733.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentence_tokens.append(text_to_word_sequence(row[0]))


[['\x00\x00\x00\x01bud1\x00\x00\x10\x00\x00\x00\x08\x00\x00\x00\x10\x00\x00\x00\x00\x86\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x10\x00\x001\x000\x002\x000dscl\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x08\x002\x000\x000\x006\x001\x000\x002\x000dsclbool\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

## **3. Creación de vectores (word2vec)**

Para crear los vectores se usa el método Word2Vec. Esta técnica transforma palabras en vectores numéricos densos (embeddings) que capturan sus significados semánticos. Funciona utilizando una red neuronal superficial y puede implementarse de dos maneras principales: CBOW (Continuous Bag of Words) y Skip-Gram.

- CBOW (Continuous Bag of Words): Predice una palabra en base a su contexto (palabras circundantes).
- Skip-Gram: Predice las palabras circundantes dado una palabra específica.

En este caso se utiliza la estructura modelo Skip-Gram:

In [7]:
#-----------------
# Creando el modelo generador de vectores
#-----------------
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,# dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

Antes de comenzar el entrenamiento, se usa también la función `build_vocab` de gensim para preparar el vocabulario con las palabras esenciales presentes en los documentos y que serán empleadas para entrenar el modelo:

In [8]:
#-----------------
# Obteniendo el vocabulario con los tokens
#-----------------
w2v_model.build_vocab(sentence_tokens)

In [9]:
#-----------------
# Cantidad de filas/docs encontradas en el corpus
#-----------------
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 53260


In [10]:
#-----------------
# Cantidad de palabras encontradas en el corpus
#-----------------
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 50855


## **4. Entrenamiento de embeddings**

Durante el entrenamiento, gensim por defecto no informa el "loss" en cada época. Por este motivo, es necesario sobrecargar el `callback` para poder tener esta información:

In [6]:
class callback(CallbackAny2Vec):
    """
    Callback para impirmir el "loss" despues de cada época
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

A continuación, se entrena el modelo generador de vectores utilizando el `callback` (esto puede tomar varios minutos):

In [12]:
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=5,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 92449536.0
Loss after epoch 1: 41768192.0
Loss after epoch 2: 0.0
Loss after epoch 3: 0.0
Loss after epoch 4: 0.0


(113160316, 139672025)

## **5. Ensayos**

Luego de entrenar el modelo, se realizan ensayos con la función `most_similar` para determinar si el modelo logró encontrar las relaciones semánticas entre las palabras del vocabulario.

Esto se logra calculando las distancias (normalmente usando la similitud coseno) entre el vector de la palabra dada y los vectores de todas las demás palabras en el vocabulario:

In [13]:
#-----------------
# Palabras que MÁS se relacionan con...:
#-----------------
w2v_model.wv.most_similar(positive=["stock"], topn=10)

[('shares', 0.5194112658500671),
 ('nsei', 0.4669107496738434),
 ('c2', 0.45477092266082764),
 ("kos'", 0.4489016532897949),
 ('mxx', 0.44003725051879883),
 ('dfmgi', 0.4301759600639343),
 ('relisted', 0.429850697517395),
 ('share', 0.4285546541213989),
 ('ipc', 0.4195040464401245),
 ('mercantile', 0.41237595677375793)]

In [14]:
#-----------------
# Palabras que MENOS se relacionan con...:
#-----------------
w2v_model.wv.most_similar(negative=["price"], topn=10)

[('richards', 0.09457418322563171),
 ('administrators', 0.08903723210096359),
 ('dick', 0.08127431571483612),
 ('overtook', 0.08123387396335602),
 ('amf', 0.06988001614809036),
 ('surgery', 0.06762833893299103),
 ('rescued', 0.06672806292772293),
 ('forensic', 0.06509034335613251),
 ('rory', 0.06370410323143005),
 ('baseball', 0.0633876621723175)]

In [15]:
#-----------------
# Palabras que MÁS se relacionan con...:
#-----------------
w2v_model.wv.most_similar(positive=["company"], topn=10)

[("company's", 0.5496138334274292),
 ('firm', 0.5337861180305481),
 ('companies', 0.505142867565155),
 ('automaker', 0.48371556401252747),
 ('retailer', 0.45829781889915466),
 ('drugmaker', 0.4575865864753723),
 ('conglomerate', 0.4315710663795471),
 ('gizmodo', 0.4299999475479126),
 ('it', 0.4272962510585785),
 ('bowes', 0.4196701943874359)]

In [16]:
#-----------------
# Palabras que MÁS se relacionan con...:
#-----------------
w2v_model.wv.most_similar(positive=["money"], topn=5)

[('funds', 0.4984147548675537),
 ('imprudently', 0.43812182545661926),
 ('cash', 0.43145644664764404),
 ('billions', 0.4286368787288666),
 ('yieldplus', 0.4285021126270294)]

In [17]:
#-----------------
# Ensayo con una palabra que no está en el vocabulario:
#-----------------
w2v_model.wv.most_similar(negative=["diedaa"])

KeyError: "Key 'diedaa' not present in vocabulary"

Los ensayos anteriores muestran que el modelo logra obtener relaciones semánticas entre las palabras del vocabulario:

- Como era esperado, palabras como *stock* se relacionan fuertemente con nombres de índices bursátiles como el *nsei* (*National Stock Exchange Index of India*) o el *dfmgi* (*Dubai Financial Market General Index*), así como con palabras relacionadas con el precio de las acciones (*share*) u otros índices como el ´ndice de precios al consumidor (*ipc*). Algo semejante se observa para el caso de la palabra *company*.

- En todos estos ejemplos donde las palabras tienen relación semántica, se observa una similitud relativamente alta, mayor a 0.4.

- Por otra, parte, cuando las palabras no guardan relación semántica, se aprecian similitudes por debajo de 0.1, como en el caso de la palabra *price*.

- Si la palbra no existe en el vocabulario, se obtiene un error del tipo `KeyError`.

Las relaciones semánticas también pueden ser detectadas usando los vectores de las palabras. En este caso se usa la función `get_vector` para obtener dicho vector y luego compararlo con los demás en el espacio vectorial:

In [19]:
#-----------------
# Obteniendo el vector de una palabra del vocabulario
#-----------------
test_vector = w2v_model.wv.get_vector("stock")
print(test_vector)

[-1.29318880e-02  2.70732582e-01 -1.15876429e-01  2.51179695e-01
 -1.03292903e-02 -3.41146618e-01  1.69383302e-01  3.20231795e-01
  1.57125980e-01 -1.77468777e-01  9.13886055e-02 -7.51681253e-02
  7.99276605e-02 -5.14600158e-01 -5.65729477e-02  1.08604498e-01
  4.97303829e-02  3.32074054e-02 -1.41999453e-01  2.67436415e-01
 -8.44132677e-02  3.91136289e-01  2.01780289e-01 -2.84428656e-01
  2.36368790e-01 -1.16444901e-02 -1.87983319e-01  5.15494160e-02
 -3.32464799e-02  1.35579124e-01 -2.22818092e-01  4.08479005e-01
 -1.91358268e-01  3.79048213e-02  1.00395299e-01 -2.19179150e-02
  5.50136082e-02  3.96489836e-02 -2.90329754e-01 -6.74729496e-02
 -1.03244275e-01  1.26001954e-01 -4.57042992e-01 -5.98676562e-01
  8.91775712e-02 -3.34297493e-02 -6.54989183e-02  2.96486586e-01
  1.59359813e-01  2.50381529e-01 -4.53924537e-01 -2.16024280e-01
  1.37689695e-01  6.48303702e-02 -2.43325248e-01  2.39879131e-01
 -9.53095332e-02  7.00446144e-02  1.55487701e-01 -7.03675076e-02
  1.26552684e-02  1.52620

El método `most_similar` también permite comparar palabras a partir de sus vectores:

In [20]:
#-----------------
# Comparando vectores
#-----------------
w2v_model.wv.most_similar(test_vector)

[('stock', 1.0),
 ('shares', 0.5194112658500671),
 ('nsei', 0.46691077947616577),
 ('c2', 0.4547709822654724),
 ("kos'", 0.4489017128944397),
 ('mxx', 0.44003725051879883),
 ('dfmgi', 0.4301759898662567),
 ('relisted', 0.429850697517395),
 ('share', 0.4285546541213989),
 ('ipc', 0.4195040464401245)]

Como se aprecia en este último ensayo, los resultados son los mismos que los obtenidos al comparar palaras directamente.

## **6. Visualización de la agrupación de vectores**

Para observar más claramente cómo se agrupan las palabras según su relación semántica, se pueden usar técnicas de reducción de dimensionalidad de vectores para graficar los embeddings en 2 y 3 dimensiones.

Esto es posible implementando funciones como la que se presenta a continuación:

In [21]:
#-----------------
# Función de reducción de dimensionalidad de los embeddings
#-----------------
def reduce_dimensions(model, num_dimensions = 2 ):
    #-----------------
    # Extrayendo vectores y etiquetas del modelo
    #-----------------
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  
    
    #-----------------
    # Reduciendo la dimensionalidad con t-SNE
    #-----------------
    tsne = TSNE(n_components=num_dimensions, random_state=0)

    #-----------------
    # Transformando los vectores al espacio reducido
    #-----------------
    vectors = tsne.fit_transform(vectors)

    #-----------------
    # Devolviendo vectores reducidos y sus etiquetas
    #-----------------
    return vectors, labels

La función `reduce_dimensions` utiliza la técnica de reducción de dimensionalidad t-SNE (t-Distributed Stochastic Neighbor Embedding) para proyectar los vectores de palabras en un espacio de menor dimensión (por defecto, 2 dimensiones).

En espacios de alta dimensión (espacio de embeddings), t-SNE calcula probabilidades de similitud entre puntos basadas en distribuciones gaussianas.

En el espacio de baja dimensión (donde se quiere graficar), se calculan probabilidades similares, pero utilizando distribuciones t de Student con un único grado de libertad, lo que ayuda a mantener una estructura más clara.

Luego, t-SNE minimiza la divergencia entre las distribuciones de similitud en los dos espacios.

Despues de reducir las dimensiones de los vectores, se puede usar un gráfico de dispersión para visualizar sus relaciones en 2D:

In [23]:
#-----------------
# Reduciendo los embedddings a 2D
#-----------------
vecs, labels = reduce_dimensions(w2v_model)

#-----------------
# Graficando los embeddings reducidos
#-----------------
MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
# fig.show(renderer="colab") # esto para plotly en colab
fig.show()

o incluso para visualizar dichas relaciones en 3D:

In [24]:
#-----------------
# Reduciendo los embedddings a 3D
#-----------------
vecs, labels = reduce_dimensions(w2v_model,3)

#-----------------
# Graficando los embeddings reducidos
#-----------------
fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
#fig.show(renderer="colab") # esto para plotly en colab
fig.show()

El gráfico en 2D muestra que:

- en el primer cuadrante se observan palabras relacionadas con agentes vinculados en el sector financiero como compañías, gobiernos, el fed, directores ejecutivos, etc. También hay referencias a capital e inversiones. Sin embargo, aquí los puntos son más dispersos, indicando una menor correlación debido, posiblemente, a su presencia en contextos amplios y diversos.
- en el segundo cuadrante se concentra un mayor número de palabras relacionadas con movimientos del mercado como *up*, *down*, *high*, *low*, *deal*,días de la semana, débitos, créditos, condicionales (*if*, *when*, *or*, *any*), algunos verbos relacionados con dichos movimientos y algunos agentes como *industry*, *bank*, *analyst* que también hacen parte de estas dinámicas.
- en el tercer cuadrante también hay una alta concentración de palabras que también guardan una cierta relación con las palabras del cuadrante anterior. Sin embargo, aquí se observan referencias a cantidades de dinero, porcentajes, negocios e incluso algunas commodities como *oil*.
- en el cuarto cuadrante la dispersión vuelve a ser mayor y se aprecian referencias a ciertos países como China y Estados Unidos, así como a ciertas monedas como el dolar y el euro.

In [ ]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)